In [ ]:
import socket
from cryptography.fernet import Fernet

# Store valid users (username: password)
users = {"user1": "password123", "admin": "adminpass"}

# Create and bind the server socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))

print("UDP Server is running...")

while True:
    # Receive username and password
    username, client_address = server_socket.recvfrom(2048)
    password, _ = server_socket.recvfrom(2048)

    username = username.decode()
    password = password.decode()

    # Authenticate user
    if username in users and users[username] == password:
        server_socket.sendto(b"Access Granted", client_address)
        print(f"User {username} connected!")

        # Receive encryption key
        key, _ = server_socket.recvfrom(2048)
        cipher = Fernet(key)

        while True:
            # Receive encrypted message
            encrypted_message, _ = server_socket.recvfrom(2048)
            decrypted_message = cipher.decrypt(encrypted_message).decode()
            print(f"{username}: {decrypted_message}")

            # Send response
            reply = input("Server: ")
            encrypted_reply = cipher.encrypt(reply.encode())
            server_socket.sendto(encrypted_reply, client_address)
    else:
        server_socket.sendto(b"Access Denied", client_address)


UDP Server is running...


In [ ]:
import socket
import requests
from cryptography.fernet import Fernet

# Create a UDP socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65433)

# Input credentials
username = input("Enter Username: ")
password = input("Enter Password: ")

# Send credentials
client_socket.sendto(username.encode(), server_address)
client_socket.sendto(password.encode(), server_address)

# Receive authentication response
response, _ = client_socket.recvfrom(2048)

if response.decode() == "Access Granted":
    print("Login successful! Secure chat initiated.")

    # Generate encryption key
    key = Fernet.generate_key()
    cipher = Fernet(key)

    # Send encryption key to server
    client_socket.sendto(key, server_address)

    while True:
        # Fetch weather data
        api_url = "https://api.open-meteo.com/v1/forecast?latitude=51.47&longitude=-0.0363&current_weather=true"
        response = requests.get(api_url)

        if response.status_code == 200:
            weather_data = response.json()
            temperature = weather_data["current_weather"]["temperature"]
            message = f"Current temperature: {temperature}°C"
        else:
            message = "Failed to fetch weather data"

        # Encrypt and send message
        encrypted_message = cipher.encrypt(message.encode())
        client_socket.sendto(encrypted_message, server_address)

        # Receive and decrypt server response
        encrypted_reply, _ = client_socket.recvfrom(2048)
        print(f"Server: {cipher.decrypt(encrypted_reply).decode()}")
else:
    print("Login failed. Exiting.")

client_socket.close()
